In [3]:
import os
import random
import string
import pandas as pd
import shutil

# 輸入影片所在的資料夾
video_folder = r"D:\works\Data_analysis\projects\balance_beam\2023May\preprocessed_DLC_data"

# 取得影片檔案清單
video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]

# 洗牌並產生檔名代號
shuffled_files = random.sample(video_files, len(video_files))
file_codes = [''.join(random.choices(string.ascii_lowercase, k=3)) for _ in range(len(video_files))]

# 建立新資料夾
output_folder = os.path.join(video_folder, "shuffled_video")
os.makedirs(output_folder, exist_ok=True)

# 複製並重新命名影片到新資料夾
mapping_data = []
for original_file, file_code in zip(shuffled_files, file_codes):
    new_file = f"{file_code}.mp4"
    shutil.copyfile(os.path.join(video_folder, original_file), os.path.join(output_folder, new_file))
    mapping_data.append((original_file, new_file))

# 儲存對應關係到CSV檔案
mapping_df = pd.DataFrame(mapping_data, columns=["original file name", "code"])
mapping_csv = os.path.join(output_folder, "mapping.csv")
mapping_df.to_csv(mapping_csv, index=False)

print("洗牌並代號化完成，影片已複製到新資料夾。")
print("對應關係已儲存至 mapping.csv 檔案中。")


洗牌並代號化完成，影片已複製到新資料夾。
對應關係已儲存至 mapping.csv 檔案中。


In [1]:
import pandas as pd

# 讀取原檔名與代碼對應的CSV檔案
mapping_df = pd.read_csv(r"D:\works\Data_analysis\projects\balance_beam\2023May\preprocessed_DLC_data\shuffled_video\mapping.csv")

# 讀取實驗數據的CSV檔案
data_df = pd.read_csv(r"G:\works\Data_analysis\projects\balance_beam\2023May\normal_walk.csv")

# 將代碼替換為原檔名
data_df = data_df.merge(mapping_df, on='code', how='left')
data_df.rename(columns={'original file name': 'original_file_name'}, inplace=True)

# 提取日期和處理方式的資訊
data_df['日期'] = data_df['original_file_name'].str.extract(r'(\d{4})')
data_df['處理方式'] = data_df['original_file_name'].str.extract(r'_([A-Za-z]+)_')

# 建立Excel寫入器
writer = pd.ExcelWriter('grouped_balance_beam_normal_walk_data.xlsx')

# 分組寫入不同分頁
for (date, treatment), group in data_df.groupby(['日期', '處理方式']):
    sheet_name = f"{date}_{treatment}"
    group.to_excel(writer, sheet_name=sheet_name, index=False)

# 保存Excel文件
writer.save()

